In [47]:
#import libraries

import os
import sys
import numpy as np
import torchtext
import pandas as pd
import re

from lib import data_utils 

#get path
src_path = os.getcwd()

datapath = src_path + '/Data/'
PLpath = datapath + 'AnnotatedPLData/PLCoref'
CENpath = datapath + 'AnnotatedCENData/CENCoref'
ONpath = datapath + 'AnnotatedONData/ONCoref'

PLInter = src_path + '/output/IntermediateFilesPL/'

#import data

#create empty data frame
PL_data = pd.DataFrame(columns=['corpusID', 'character', 'animacy', 'coref_chain', 'chain_head', 'head_of_head', 'chain_len', 'CL'])

#append all PL texts and features into one dataframe
for n in range(1,47):

    #get story path
    storypath = PLpath+'/story' + str(n) + '.txt'

    #read in story
    corpus = data_utils.read_story(storypath)

    #list of features
    features = ["CN", "Dep", "NER", "SS", "Triple", "WN"]

    for f in features:
        #empty list
        feat = []
        with open(PLInter + f + 'FeatureBoolean'+'/Story' + str(n) + '.txt', 'r') as doc:
            for line in doc:
                feat.append(eval(line.rstrip()))
    
        corpus[f] = feat

    #append to dataframe
    PL_data = pd.concat([PL_data, corpus], ignore_index=True)

PL_data

,corpusID,character,animacy,coref_chain,chain_head,head_of_head,chain_len,CL,CN,Dep,NER,SS,Triple,WN
0,story1,1,1,"[ dragon , he , he , the dragon , He , hi...",dragon,dragon,43,2.499540,1.0,1.0,0.0,1.0,1.0,0.0
1,story1,1,1,"[ princess , the tsar 's daughter , her , h...",princess,princess,23,0.990763,1.0,1.0,0.0,1.0,0.0,1.0
2,story1,1,1,"[ tsar , tsar , father , tsar , her father...",tsar,tsar,9,-0.065380,1.0,1.0,0.0,1.0,1.0,1.0
3,story1,0,1,"[ princess' dog , a little dog that had follo...",princess' dog,dog,4,-0.442574,1.0,1.0,0.0,1.0,1.0,0.0
4,story1,1,1,"[ tsarina , mother , tsarina , tsarina ]",tsarina,tsarina,4,-0.442574,1.0,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1907,story46,0,1,"[ the entire enemy army , the enemy army , h...",the entire enemy army,army,7,-0.308434,0.0,1.0,0.0,0.0,0.0,0.0
1908,story46,0,1,"[ this , the best solution ]",this,this,2,-0.604613,0.0,0.0,0.0,1.0,0.0,0.0
1909,story46,0,1,"[ a horse , a horse ]",a horse,horse,2,-0.604613,0.0,0.0,0.0,0.0,1.0,0.0
1910,story46,0,1,"[ an even better horse , his horse , his hor...",an even better horse,horse,4,-0.486142,0.0,0.0,0.0,0.0,1.0,0.0


In [82]:
# simple model 

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

data_y = PL_data["character"].astype('int')
data_x = PL_data[["CL", "CN", "Dep", "NER", "SS", "Triple", "WN"]]

X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, random_state=0)

rbf_svc = svm.SVC(kernel = 'rbf', C=.5, gamma=1)

cv_results = cross_validate(rbf_svc, X_train, y_train, cv = 10, scoring=('f1', 'accuracy'), return_train_score=True)

print(cv_results['train_accuracy'])
print(cv_results['test_accuracy'])
print(cv_results['train_f1'])
print(cv_results['test_f1'])

# cv = StratifiedShuffleSplit(n_splits=10, test_size=.2, random_state=42)

# C_range = np.logspace(-2,10,13)
# gamma_range = np.logspace(-9,3,13)

# grid = GridSearchCV(svm.SVC(kernel='rbf'), param_grid=dict(gamma = gamma_range, C= C_range), cv=cv)

# grid.fit(X_train,y_train)

# print(grid.best_params_, grid.best_score_)



[0.90775194 0.9124031  0.91007752 0.91007752 0.90859799 0.91402014
 0.9070488  0.91092177 0.91324555 0.91247095]
[0.92361111 0.875      0.90972222 0.90277778 0.90909091 0.8951049
 0.92307692 0.91608392 0.86713287 0.88811189]
[0.82212257 0.8320951  0.82789318 0.82738095 0.82440476 0.83308271
 0.82035928 0.82962963 0.83577713 0.83109118]
[0.86419753 0.75675676 0.82191781 0.82051282 0.82191781 0.81012658
 0.85333333 0.83783784 0.71641791 0.78947368]


In [3]:
from allennlp.predictors.predictor import Predictor
from allennlp_models import pretrained
import allennlp_models.tagging
# print(pretrained.get_pretrained_models())


# predictor_ss = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/structured-prediction-srl-bert.2020.12.15.tar.gz")
predictor_ner = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/ner-elmo-2021.02.12.tar.gz")

results = predictor_ner.predict_instance("Did Uriah honestly think he could beat The Legend of Zelda in less than three hours?")


KeyboardInterrupt: 